In [1]:
import os
import sys 
import configparser
import pandas as pd
import numpy as np



config = configparser.ConfigParser()
config.read('../config.ini')
data_folder = config['paths']['path_data']

KeyError: 'paths'

### SQL queries 

In [2]:
import pyodbc





query_symbols_correspondence = """
        SELECT [sym_sedol_hist].[fsym_id]
              ,[bbg_ticker]
              ,[sedol]
              ,[fsym_security_id]
              ,[ce_sec_entity].factset_entity_id
              ,factset_ultimate_parent_entity_id
        FROM [fds].[sym_v1].[sym_sedol_hist]

        join [fds].[sym_v1].[sym_coverage]
        on [sym_coverage].fsym_id = [sym_sedol_hist].[fsym_id]

 

        join [fds].[sym_v1].[sym_bbg]
        on sym_bbg.fsym_id = sym_sedol_hist.fsym_id

 

        join [fds].[evt_v1].[ce_sec_entity]
        on [fsym_security_id] = [ce_sec_entity].fsym_id

 

        join [fds].[ent_v1].[ent_entity_structure]
        on [ce_sec_entity].factset_entity_id = [ent_entity_structure].factset_entity_id
    """

query_beta = """
SELECT  px.fsym_id
      , px.p_date
      , px.exch_rate_sec_eur
      , px.p_price_adj_split_spinoff_div
      , px.mcap_local
      ,sec.factset_industry
      ,sec.factset_sector
	  ,[04_Factor_Raw_Histo].[factor_value] as beta

  FROM [ScreenFactory].[dbo].[03_DailyPx_Temp] px

  left join [ScreenFactory].[dbo].[Securities_Fds] sec
  on px.fsym_id = sec.fsym_id

  left join [ScreenFactory].[dbo].[04_Factor_Raw_Histo] 
  on px.fsym_id = [04_Factor_Raw_Histo].fsym_id AND px.p_date = [04_Factor_Raw_Histo].p_date

  WHERE factor_raw_id = 255 
  order by px.p_date desc

"""

### Price Data

In [3]:
os.listdir(os.path.join(data_folder))

['news_200304_202212.pkl',
 'news_headlines_final_v2.1(raw).parquet',
 'news_headlines_v1.1.parquet',
 'news_headlines_final_v2_merged.parquet',
 'news_headlines_final_v1.2.parquet',
 'news_headlines_v1.2.parquet',
 'news_headlines_v3.parquet',
 'df_price.parquet',
 'news_headlines_v1.2_notmerged.parquet',
 'trscript_data_t_199905_202205.pkl',
 'news_headlines_final_v2.parquet',
 'df_price_v1.1.parquet',
 'news_headlines_v3.2.parquet']

In [4]:
import pyodbc
import pandas as pd

query_symbols_correspondence = """
        SELECT [sym_sedol_hist].[fsym_id]
              ,[bbg_ticker]
              ,[sedol]
              ,[fsym_security_id]
              ,[ce_sec_entity].factset_entity_id
              ,factset_ultimate_parent_entity_id
        FROM [fds].[sym_v1].[sym_sedol_hist]

        join [fds].[sym_v1].[sym_coverage]
        on [sym_coverage].fsym_id = [sym_sedol_hist].[fsym_id]

 

        join [fds].[sym_v1].[sym_bbg]
        on sym_bbg.fsym_id = sym_sedol_hist.fsym_id

 

        join [fds].[evt_v1].[ce_sec_entity]
        on [fsym_security_id] = [ce_sec_entity].fsym_id

 

        join [fds].[ent_v1].[ent_entity_structure]
        on [ce_sec_entity].factset_entity_id = [ent_entity_structure].factset_entity_id
    """

# Your connection string
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=RAMASRVSQL03.ramai.local;"
    "UID=linuxml;"
    "PWD=lymph-nBFKGPj5"
)
conn_str

# Initiate the connection
conn = pyodbc.connect(conn_str)

# Execute the query and read into a DataFrame
df_symbols = pd.read_sql(query_symbols_correspondence, conn)

# Close the connection
conn.close()

# Show the DataFrame
df_symbols.head(2)

/tmp/ipykernel_4146257/3186477113.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_symbols = pd.read_sql(query_symbols_correspondence, conn)


,fsym_id,bbg_ticker,sedol,fsym_security_id,factset_entity_id,factset_ultimate_parent_entity_id
0,B06864-R,None,BJXPN49,KRL7VP-S,05VJB4-E,05VJB4-E
1,B0BPPB-R,CUBE* MM,BMGSK78,QHD9GG-S,0067BK-E,0067BK-E


In [5]:
list(filter(lambda st : 'price' in st, os.listdir(os.path.join(data_folder))))

['df_price.parquet', 'df_price_v1.1.parquet']

In [6]:
import pandas as pd
import numpy as np
import os 
import datetime, re


df_price = pd.read_parquet(os.path.join(data_folder, 'df_price_v1.1.parquet'))
df_price = df_price.rename(columns=  {'mcap_local': 'DES_MCAP_CCY', 'factset_industry': 'DES_FACTSET_INDUSTRY', 'factset_sector': 'DES_FACTSET_SECTOR'})


In [7]:
for col in df_price.columns:
    print(col)

exch_rate_sec_eur
p_price_adj_split_spinoff_div
DES_MCAP_CCY
DES_FACTSET_INDUSTRY
DES_FACTSET_SECTOR
DES_BETA_ADJ_104W
RET_5D_bin
RET_10D_bin


In [16]:

news_pkl = pd.read_pickle(os.path.join(data_folder, 'news_200304_202212.pkl'))
news_pkl.head(2)
for col in news_pkl.columns:
    print(col)

all_factset_entity_id
all_ticker_region
factset_entity_id
fds_country
fds_language
fds_subject
headline
headline_no_ent
last_update_time_stamp
product
publish_time
reference_story_headline
reference_story_id
reference_story_time
report_id
sa_category
schema_version
senti_headline
senti_headline_no_ent
story_body
story_date_time
ticker_region


In [17]:
cols_date = [col for col in news_pkl.columns if "date" in col.lower() or "time" in col.lower()]
cols_date
news_pkl[cols_date].iloc[:1000:100]

,last_update_time_stamp,publish_time,reference_story_time,story_date_time
0,None,2012-06-06T05:40:39Z,[],2003-10-01T00:58:49Z
100,None,2012-06-06T05:45:26Z,[],2003-10-01T12:09:55Z
200,None,2012-06-06T05:51:49Z,[],2003-10-01T18:37:50Z
300,None,2012-06-06T07:28:01Z,[],2003-10-10T11:07:16Z
400,None,2012-06-06T07:34:46Z,[],2003-10-10T17:45:00Z
500,None,2012-06-06T07:45:18Z,[],2003-10-13T11:11:30Z
600,None,2012-06-06T07:50:42Z,[],2003-10-13T16:47:51Z
700,None,2012-06-06T07:59:30Z,[],2003-10-14T10:19:50Z
800,None,2012-06-06T08:02:13Z,[],2003-10-14T12:40:43Z
900,None,2012-06-06T08:09:16Z,[],2003-10-14T20:08:07Z


In [18]:
news_pkl.columns

Index(['all_factset_entity_id', 'all_ticker_region', 'factset_entity_id',
       'fds_country', 'fds_language', 'fds_subject', 'headline',
       'headline_no_ent', 'last_update_time_stamp', 'product', 'publish_time',
       'reference_story_headline', 'reference_story_id',
       'reference_story_time', 'report_id', 'sa_category', 'schema_version',
       'senti_headline', 'senti_headline_no_ent', 'story_body',
       'story_date_time', 'ticker_region'],
      dtype='object')

In [19]:
#create SYMBOL index
print(f"creating SYMBOL index")
news_pkl = news_pkl.rename(columns = {"factset_entity_id": "SYMBOL", "story_date_time": "DATE"})
news_pkl = news_pkl.explode("SYMBOL").dropna(subset = ['SYMBOL']) # converts list of symbols into separate rows
news_pkl = news_pkl[['DATE', 'SYMBOL', 'headline_no_ent']]
news_pkl.head(10)


creating SYMBOL index


,DATE,SYMBOL,headline_no_ent
5,2003-10-01T02:30:05Z,000Q29-E,reaffirmed outlook for 12-14% EPS growth in 2004
9,2003-10-01T09:13:42Z,00118C-E,initiated neutral at
14,2003-10-01T09:20:09Z,000MZR-E,initiated underweight at
15,2003-10-01T09:23:59Z,000FGN-E,upgraded to equal weight from underweight at
18,2003-10-01T09:29:49Z,002Q0G-E,"could face new payments from spill, reports WSJ"
21,2003-10-01T09:35:10Z,002RBQ-E,upgraded to overweight from neutral at
22,2003-10-01T09:36:14Z,00C4H1-E,"expected to introduce a service today, repor..."
23,2003-10-01T09:38:56Z,003FY6-E,upgraded to sector perform from underperform at
24,2003-10-01T09:39:56Z,0027L0-E,C CEO takes over helm today; plans to concent...
26,2003-10-01T09:48:10Z,0091M7-E,upgraded to buy from neutral at


In [12]:
from itertools import islice
with open("news_headlines_v3.2_intermediate.txt","w") as f:
    f.write("headline_no_ent|headline\n")
    for i, row in islice(news_pkl.iterrows(),2000):
        f.write(f"{row['headline_no_ent']}|{row['headline']}\n")

In [20]:
# process DATE : convert 2012-06-06T05:40:39Z to 2012-06-06
featcol = "headline_no_ent"
print(f"processing DATE")
datecol = pd.to_datetime(news_pkl['DATE']).dt.tz_localize(None).dt.strftime('%Y-%m-%d')
news_pkl['DATE'] = pd.to_datetime(datecol)

#set the index to SYMBOL, DATE
print(f"setting index to SYMBOL, DATE")
news_pkl = news_pkl.set_index(['SYMBOL', 'DATE']).sort_index()

print(f"shifting dates by 1 day")
news_pkl['DATE2'] =( news_pkl.reset_index(level = 1)['DATE'] + datetime.timedelta(days=1)).values
news_pkl = news_pkl.reset_index(level = 1, drop = True).rename(columns = {"DATE2": "DATE"}).reset_index(drop=False).set_index(['SYMBOL', 'DATE']).sort_index()

print('removing "StreetAccount" reports ')
news_pkl.filter(regex = '.*Street\s*Account.*', axis = 0)
def filter_function(x:str):
    pattern = r'.*Street\s*Account.*'
    if not x.__class__ == str:
        return False
    return re.search(pattern, x) is not None
news_pkl['hasSA'] = news_pkl[featcol].apply(filter_function)
print("num 'StreetAccount' rows : ", news_pkl.hasSA.sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])


processing DATE
setting index to SYMBOL, DATE
shifting dates by 1 day
removing "StreetAccount" reports 
num 'StreetAccount' rows :  0.05041123352892069 out of  2679621


In [22]:
news_pkl = news_pkl.loc[~news_pkl.hasSA, [featcol]]

news_pkl[featcol] = news_pkl[featcol].str.replace(r'\bEPS\b', 'Earnings Per Share', regex=True)
news_pkl[featcol] = news_pkl[featcol].str.replace(r'[-]{1,2} [A-Z]\w*', '', regex=True)


In [24]:


def filter_function2(x:str):
    pattern = r'trading (higher|lower|up|down)\b'
    if not x.__class__ == str:
        return False
    return re.search(pattern, x.lower()) is not None
news_pkl["hasTrading"] = news_pkl[featcol].apply(filter_function2)

print("num 'Reports vs' rows : ", news_pkl["hasTrading"].sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])


num 'Reports vs' rows :  0.005559358909161506 out of  2544538


In [26]:
news_pkl = news_pkl.loc[~news_pkl["hasTrading"], [featcol]]


In [27]:

news_pkl['has_misc'] = news_pkl[featcol].str.contains(r"Barron's Summary|Street Takeaways", case=False, regex=True)


In [28]:
print(f"num rows with 'Barron's Summary|Street Takeaways' : ", news_pkl['has_misc'].sum() / news_pkl.shape[0], "out of ", news_pkl.shape[0])


num rows with 'Barron's Summary|Street Takeaways' :  0.010517737963129823 out of  2530392


In [33]:
news_pkl.isna().sum()/news_pkl.shape[0]
news_pkl = news_pkl.dropna(subset = [featcol])

In [38]:

news_pkl = news_pkl.loc[~news_pkl.has_misc.astype(bool), [featcol]]



In [39]:

news_pkl = news_pkl[news_pkl[featcol].apply(lambda x: len(x.split()) > 5)]


In [40]:
print(f"number of symbols : {len(news_pkl.index.get_level_values(0).unique())}")
print(f"number of dates : {len(news_pkl.index.get_level_values(1).unique())}")
print(f"number of headlines : {news_pkl.shape[0]} ie {news_pkl.shape[0] / 10**6 :.2f}M")

number of symbols : 23876
number of dates : 6840
number of headlines : 1859531 ie 1.86M


In [41]:

print(f"merging with df_symbols")
# merge -E and -R, and keep -R, but one -R can have multiple -Es
news_pkl= news_pkl.reset_index(drop=False).merge(df_symbols[['fsym_id', 'factset_entity_id']], left_on = 'SYMBOL', right_on = 'factset_entity_id', how = 'left')
del news_pkl['SYMBOL']
news_pkl.rename(columns = {"fsym_id": "SYMBOL"}, inplace = True)
if not "SYMBOL" in news_pkl.index.names and not "DATE" in news_pkl.index.names:
    news_pkl = news_pkl.set_index(['SYMBOL', 'DATE']).sort_index()

print(f"merging with df_price")
cols_final = ['SYMBOL', 'DATE', 'RET_5D_bin',  'RET_10D_bin','DES_MCAP_CCY','DES_FACTSET_INDUSTRY',	'DES_FACTSET_SECTOR'	,'DES_BETA_ADJ_104W']
out = news_pkl[[featcol]].merge(
    df_price.reset_index(drop=False)[cols_final], 
    on = ['SYMBOL', 'DATE'],
    how = 'left').dropna(subset = ['RET_10D_bin', 'DES_BETA_ADJ_104W'])
diff = set(news_pkl.index.get_level_values(0)) - set(df_price.index.get_level_values(0))
print(f"number of NA symbols : {len(diff)} out of {len(set(news_pkl.index.get_level_values(0)))}")



merging with df_symbols
merging with df_price
number of NA symbols : 64288 out of 79740


In [42]:
out.shape
print(f"number of dates {out.DATE.nunique()} and SYMBOLs {out.SYMBOL.nunique()}")
print(f"min date {out.DATE.min()} and max date {out.DATE.max()}")
print(f"shape {out.shape}")

number of dates 4324 and SYMBOLs 10928
min date 2003-04-11 00:00:00 and max date 2019-12-31 00:00:00
shape (1392046, 9)


In [43]:
out.to_parquet(os.path.join(data_folder, 'news_headlines_v3.3.parquet'))

In [54]:
def gen():
    for i in range(0, out.shape[0]):
        yield out.iloc[i]['RET_10D_bin'],out.iloc[i]['headline']
gene = gen()


In [45]:

out = out.reset_index(drop=False).drop_duplicates(subset = ['SYMBOL','DATE',featcol, 'RET_5D_bin', 'RET_10D_bin'], keep = 'first')

In [46]:
del out['index']


In [47]:
if "SYMBOL" in out.columns and "DATE" in out.columns:
    out = out.set_index(['SYMBOL', 'DATE']).sort_index()
out.iloc[2000:2050:2]

headline_no_ent  \
SYMBOL   DATE                                                            
B19ST9-R 2010-08-18  Follow-up x2:  halts development of semagacest...   
         2010-08-20  Follow-up:  panel votes 8-5  is effective in t...   
         2010-08-20   panel votes 9-4 that  data does not show  eff...   
         2010-08-20   panel votes 8-6 to recommend expanded indicat...   
         2010-09-02  Follow-up:  affirms lower court ruling that ha...   
         2010-09-02  Follow-up:  grants  motion for an injunction b...   
         2010-09-02   issues press release regarding  upholding the...   
         2010-10-12   presents phase 3 trial data evaluating  misse...   
         2010-10-22   reports Q3 Earnings Per Share $1.21 ex-items ...   
         2010-11-05    approves  to treat chronic musculoskeletal pain   
         2010-11-17   says new results from substudy from  indicate...   
         2010-12-02           panel to consider  at January 12 meeting   
         2010-12-21   announces completion of  and  priority review...   
         2011-01-11   releases briefing documents for panel meeting...   
         2011-01-13                     upgraded to hold from sell at    
         2011-01-13   says patient in solanezumab study developed b...   
         2011-01-19  Follow-up:  releases briefing documents for pa...   
         2011-01-21  Follow-up:  votes that existing data does not ...   
         2011-01-21   concurs that knowledge of brain amyloid plaqu...   
         2011-01-28   reports Q4 Earnings Per Share $1.11 ex-items ...   
         2011-02-10  Preview of upcoming  decision on  (liprotamase...   
         2011-03-18   announces action regarding recall of alcohol ...   
         2011-04-05  Two novel  molecules target genetic mutations ...   
         2011-04-20   receives complete response letter from  for l...   
         2011-04-27   and  announce drug-device collaboration for  ...   

                     RET_5D_bin  RET_10D_bin  DES_MCAP_CCY  \
SYMBOL   DATE                                                
B19ST9-R 2010-08-18         1.0          0.0   40106209.20   
         2010-08-20         1.0          0.0   39402793.80   
         2010-08-20         1.0          0.0   39402793.80   
         2010-08-20         1.0          0.0   39402793.80   
         2010-09-02         1.0          1.0   39564233.40   
         2010-09-02         1.0          1.0   39564233.40   
         2010-09-02         1.0          1.0   39564233.40   
         2010-10-12         0.0          0.0   43081310.40   
         2010-10-22         0.0          0.0   40821156.00   
         2010-11-05         1.0          1.0   41190267.96   
         2010-11-17         0.0          0.0   39668119.20   
         2010-12-02         0.0          0.0   39495147.75   
         2010-12-21         0.0          0.0   40521445.02   
         2011-01-11         0.0          0.0   40256222.13   
         2011-01-13         1.0          1.0   40244690.70   
         2011-01-13         1.0          1.0   40244690.70   
         2011-01-19         1.0          1.0   39956404.95   
         2011-01-21         0.0          1.0   40083250.68   
         2011-01-21         0.0          1.0   40083250.68   
         2011-01-28         1.0          0.0   40094782.11   
         2011-02-10         0.0          0.0   41074953.66   
         2011-03-18         0.0          0.0   39916289.20   
         2011-04-05         1.0          1.0   40529851.65   
         2011-04-20         1.0          1.0   41826436.45   
         2011-04-27         1.0          1.0   43099867.95   

                       DES_FACTSET_INDUSTRY DES_FACTSET_SECTOR  \
SYMBOL   DATE                                                    
B19ST9-R 2010-08-18  Pharmaceuticals: Major  Health Technology   
         2010-08-20  Pharmaceuticals: Major  Health Technology   
         2010-08-20  Pharmaceuticals: Major  Health Technology   
         2010-08-20  Pharmaceuticals: Major  Health

In [48]:
print( f"number of dates {out.index.get_level_values(1).nunique()} and SYMBOLs {out.index.get_level_values(0).nunique()}")
print(f"min date {out.index.get_level_values(1).min()} and max date {out.index.get_level_values(1).max()}")
print(f"shape {out.shape}")

number of dates 4324 and SYMBOLs 10928
min date 2003-04-11 00:00:00 and max date 2019-12-31 00:00:00
shape (834279, 7)


In [75]:
out.to_parquet(os.path.join(data_folder, 'news_headlines_v3.parquet'))

### Changing target to binary positive negative

In [50]:
df_price = pd.read_parquet(os.path.join(data_folder, 'df_price.parquet'))
df_price.columns

Index(['exch_rate_sec_eur', 'p_price_adj_split_spinoff_div', 'DES_MCAP_CCY',
       'DES_FACTSET_INDUSTRY', 'DES_FACTSET_SECTOR', 'DES_BETA_ADJ_104W',
       'RET_5D', 'RET_5D_bin', 'RET_10D', 'RET_10D_bin'],
      dtype='object')

In [52]:
df_price['RET_5D_pos'] = (df_price['RET_5D'] > 0).astype(int)
df_price['RET_10D_pos'] = (df_price['RET_10D'] > 0).astype(int)
df_price[['RET_5D_pos', 'RET_10D_pos', 'RET_5D', 'RET_10D']].head(10)

RET_5D_pos  RET_10D_pos    RET_5D   RET_10D
SYMBOL   DATE                                                   
B00165-R 1998-01-01           1            0  0.011591 -0.056464
         1998-01-02           1            0  0.009476 -0.001882
         1998-01-05           1            1  0.000650  0.002199
         1998-01-06           1            1  0.007658  0.025806
         1998-01-07           0            0 -0.024011 -0.014027
         1998-01-08           0            0 -0.067275 -0.022539
         1998-01-09           0            0 -0.011251 -0.021632
         1998-01-12           1            0  0.001548 -0.031423
         1998-01-13           1            1  0.018010  0.028745
         1998-01-14           1            1  0.010230  0.006582

In [59]:
if "out" in locals():
    newsdf = out
else:
    newsdf = pd.read_parquet(os.path.join(data_folder, 'news_headlines_v3.parquet'))

In [60]:
newsdf.head(2)

headline_no_ent  \
SYMBOL   DATE                                                            
B00FG1-R 2016-01-13   guides FY (Dec 2016) Revenue $143.5 vs FactSe...   
         2016-02-03   acquires four refined product terminals from ...   

                     RET_5D_bin  RET_10D_bin  DES_MCAP_CCY  \
SYMBOL   DATE                                                
B00FG1-R 2016-01-13         0.0          0.0     357317.10   
         2016-02-03         1.0          1.0     324816.94   

                       DES_FACTSET_INDUSTRY     DES_FACTSET_SECTOR  \
SYMBOL   DATE                                                        
B00FG1-R 2016-01-13  Wholesale Distributors  Distribution Services   
         2016-02-03  Wholesale Distributors  Distribution Services   

                     DES_BETA_ADJ_104W  
SYMBOL   DATE                           
B00FG1-R 2016-01-13          -0.901957  
         2016-02-03          -0.888100

In [56]:
df_price.head(2)

exch_rate_sec_eur  p_price_adj_split_spinoff_div  \
SYMBOL   DATE                                                           
B00165-R 1998-01-01           0.916674                          8.375   
         1998-01-02           0.920302                          8.125   

                     DES_MCAP_CCY DES_FACTSET_INDUSTRY   DES_FACTSET_SECTOR  \
SYMBOL   DATE                                                                 
B00165-R 1998-01-01    105131.375    Packaged Software  Technology Services   
         1998-01-02    101993.125    Packaged Software  Technology Services   

                     DES_BETA_ADJ_104W    RET_5D  RET_5D_bin   RET_10D  \
SYMBOL   DATE                                                            
B00165-R 1998-01-01          -1.531950  0.011591         1.0 -0.056464   
         1998-01-02          -1.517021  0.009476         1.0 -0.001882   

                     RET_10D_bin  RET_5D_pos  RET_10D_pos  
SYMBOL   DATE                                              
B00165-R 1998-01-01          0.0           1            0  
         1998-01-02          1.0           1            0

In [61]:
col_no_target = [col for col in newsdf.columns if "RET" not in col] + ["SYMBOL","DATE"]
newsdf = newsdf.reset_index(drop=False)[col_no_target].rename(columns = {'headline_no_ent': 'headline'})
newsdf = newsdf.merge(df_price[['RET_5D_pos', 'RET_10D_pos']], left_on = ['SYMBOL', 'DATE'], right_on = ['SYMBOL', 'DATE'], how = 'left')

In [62]:
print(newsdf.shape)
print(newsdf.dropna(subset = ['RET_5D_pos', 'RET_10D_pos']).shape)

(834279, 9)
(834279, 9)


In [63]:
data_folder

'/mnt/nvme0n1p1/ml_data_tre/LLM_project'

In [64]:
newsdf.to_parquet(os.path.join(data_folder, 'news_headlines_v3.3.parquet'))